<a href="https://colab.research.google.com/github/ujj-kum/Google-Colab-Notebooks/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [2]:
print("Train data = ", len(X_train), type(X_train), X_train.shape)
print("Test data = ", len(X_test))

Train data =  50000 <class 'numpy.ndarray'> (50000, 32, 32, 3)
Test data =  10000


In [3]:
set(y_test.reshape(10000))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [4]:
X_train = X_train/255.0
X_test = X_test/255.0

In [5]:
import keras
from keras.models import Sequential, Model
from keras.layers import Add, Activation, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, AveragePooling2D, BatchNormalization
from keras.optimizers import Adam

In [6]:
def residual_block(x, filters, downsample=False):
    strides = (2, 2) if downsample else (1, 1)

    # 1st CNN Layer of the block
    y = Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    # 2nd CNN layer of the block
    y = Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
    y = BatchNormalization()(y)

    # Skip connection if downsample or number of filters change
    # x output size changes, so we make x dowsample to the same size
    if downsample:
        # Not for feature Engineering, just size reduction
        x = Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(x)

    # Add skip connection
    y = Add()([x, y])
    y = Activation('relu')(y)
    return y

In [7]:
def resNet(input_shape, num_classes=10):
    inputs = keras.Input(shape=input_shape)

    # Initial Conv2D Layer
    x = Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Residual Blocks
    x = residual_block(x, filters=16)
    x = residual_block(x, filters=16)
    x = residual_block(x, filters=32, downsample=True)
    x = residual_block(x, filters=32)
    x = residual_block(x, filters=64, downsample=True)
    x = residual_block(x ,filters=64)

    x = AveragePooling2D(pool_size=(8, 8))(x)

    x = Flatten()(x)

    outputs=Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


### Build and train the model

In [8]:
model = resNet(input_shape=(32, 32, 3), num_classes=10)

loss='sparse_categorical_crossentropy'
optimizer='adam'
metrics=['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

hist = model.fit(X_train, y_train, batch_size=64, epochs=1,
                 validation_data=(X_test, y_test),
                 shuffle=True)

782/782 [==============================] - 414s 519ms/step - loss: 1.3065 - accuracy: 0.5277 - val_loss: 1.4472 - val_accuracy: 0.5143
